In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import contractions
#import matplotlib.pyplot as plt

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.neural_network import MLPClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.svm import LinearSVC
# from sklearn.linear_model import LogisticRegressionCV
# from sklearn.linear_model import Perceptron

import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 


nltk.download('punkt', quiet = True)
nltk.download('wordnet', quiet = True)
nltk.download('averaged_perceptron_tagger', quiet = True)
nltk.download('stopwords', quiet = True)
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [71]:
! pip install zstandard

In [430]:
## using in block processing to make data visualize easier
# def preprocess_data(df):
#     # lower case
#     df['Text'] = df['Text'].str.lower()

#     # remove url
#     df['Text'] = df['Text'].replace(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?', 'URL', regex=True)

#     # remove non-alphabetical
#     df['Text'] = df['Text'].str.replace('[^a-zA-Z\']', ' ', regex=True).str.strip()


#     # remove extra spaces
#     df['Text'] = df['Text'].str.replace(' +', ' ', regex=True).str.strip()

#     # #expand contractions
#     df['Text'] = df['Text'].apply(lambda x: contractions.fix(x))
#     # df['Text'] = df['Text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
#     # #join back words
#     # df['Text'] = [' '.join(map(str, l)) for l in X['Text']]

#     # #remove html and url form reviews
#     # df['Text'] = df['Text'].str.replace(r'\s*https?://\S+(\s+|$)', '', regex=True).str.strip()
#     # #remove non-alphabetical characters
#     # df['Text'] = df['Text'].str.replace('[^a-zA-Z]', ' ', regex=True)
#     # #remove extra spaces
#     # df['Text'] = df['Text'].replace(r'\s+', ' ', regex=True)
#     return df

In [2]:
def tokenize_and_remove_stopwords(X):
    stop_words = set(stopwords.words('english'))
    X['tokens'] = X['Text'].str.split()
    return X['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

In [3]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return "UNK"

In [4]:
def vectorize_tfidf(X):
    tfidf=TfidfVectorizer(min_df = 50, max_df=0.95, ngram_range = (1,3), max_features=1500, norm='l2')
    X_data = tfidf.fit_transform(X)
    return X_data.toarray()

In [10]:
#read csv file and convert to dataframe
df = pd.read_pickle('data/all_year.pkl.zst',compression = 'zstd')

In [9]:
 # remnove nan from the df
df = df.dropna(how="any")

In [7]:
#df = df[:5]

In [11]:
df

,Title,Docket,Argued,Decided,Text
0,"BROWN, ACTING WARDEN v. DAVENPORT(2022)",No. 20-826,"October 5, 2021","April 21, 2022","United States Supreme Court BROWN, ACTING WARD..."
1,"CAMERON, ATTORNEY GENERAL OF KENTUCKY v. EMW W...",No. 20-601,"October 12, 2021","March 3, 2022","United States Supreme Court CAMERON, ATTORNEY ..."
2,"CITY OF AUSTIN, TEXAS v. REAGAN NATIONAL ADVER...",No. 20-1029,"November 10, 2021","April 21, 2022","United States Supreme Court CITY OF AUSTIN, TE..."
3,FEDERAL BUREAU OF INVESTIGATION ET AL. v. FAZA...,No. 20-828,"November 8, 2021","March 4, 2022",United States Supreme Court FEDERAL BUREAU OF ...
4,HOUSTON COMMUNITY COLLEGE SYSTEM v. WILSON(2022),No. 20-804,"November 2, 2021","March 24, 2022",United States Supreme Court HOUSTON COMMUNITY ...
...,...,...,...,...,...
21866,"WHOLE WOMAN'S HEALTH ET AL. v. JACKSON, JUDGE,...",No. 21-463,"November 1, 2021","December 10, 2021",United States Supreme Court WHOLE WOMAN'S HEAL...
21867,WHOLE WOMAN'S HEALTH ET AL. v. AUSTIN REEVE ...,No. 21a24,,"September 1, 2021",United States Supreme Court WHOLE WOMAN'S HEAL...
21868,"WILLIAM P. BARR, ATTORNEY GENERAL, ET AL. v. ...",No. 20a8,,"July 14, 2020","United States Supreme Court WILLIAM P. BARR, A..."
21869,"WILLIAM P. BARR, ATTORNEY GENERAL, ET AL. v. ...",No. 20a9,,"July 16, 2020","United States Supreme Court WILLIAM P. BARR, A..."


In [12]:
def remove_space(match_obj):
    if match_obj.group() is not None:
        return match_obj.group().replace(' ','|')

In [13]:
# write a function to write text column of first row to a file
def write_to_file(filename):
    with open(filename, 'w') as f:
        f.write(str(df["tokens"][0]))

# write_to_file("data/test1.txt")

In [14]:
# Disabled as it might be important later on
# # lower case
# df['Text'] = df['Text'].str.lower()

# remove url
df['Text'] = df['Text'].str.replace(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?', 'URL', regex=True)


# remove '|' so we can use it as a unique space seperator for cases
df['Text'] = df['Text'].str.replace(r'|', '', regex=True)


df['Text'] = df['Text'].str.replace(r'U\. S\. C\.', 'U.S.C.', regex=True)
df['Text'] = df['Text'].str.replace(r'U\. S\.', 'U.S.', regex=True)
df['Text'] = df['Text'].str.replace(r'No\. ', 'No.', regex=True)
r"F\.( \d+-?\w*,?)+( \([A-Z]*\d* ?\d*\),?)*"
# df['Text'] = df['Text'].str.replace(r"F\.( \d+-?\w*,?)+( \([A-Z]*\d* ?\d*\),?)*", remove_space, regex=True)
text = []
for element in df['Text']:
    element = re.sub(r"\bF\.( \d+-?\w*,?)+( \([A-Z]*\d* ?\d*\),?)*", remove_space, element)
    element = re.sub(r"\b(\d+ )?U\.S\.C\. [A-Z]+\d*", remove_space, element)
    element = re.sub(r"\b(\d+ )?U\.S\.(( |-)\(?\d+,?\)?)+", remove_space, element)
    element = re.sub(r"\bArt\. [A-Z]+,?( [A-Z]\d+)?", remove_space, element)
    element = re.sub(r"\b\d{4} [A-Z]+ \d+,?( \*\d*)?", remove_space, element)
    element = re.sub(r"\bn\.( \d+)+( \(.+?\))?", remove_space, element)
    element = re.sub(r"\bPp\. [0-9\-]+", remove_space, element)
    # element = re.sub(r"\b([A-Z]+[a-z]*[A-Z\.']+,? )+v\.( [A-Z]+[a-z]*[A-Z\.']+,?)+ ?", remove_space, element)
    element = re.sub(r"\b\d+ U\.S.[_,\- ]*\(\d+\)", remove_space, element)
    text.append(element)
df['Text'] = text
# Disabled as legal cases need punctuations to work
# # remove non-alphabetical
# df['Text'] = df['Text'].str.replace('[^a-zA-Z0-9\'\".!()]', ' ', regex=True).str.strip()


# remove extra spaces
df['Text'] = df['Text'].str.replace(' +', ' ', regex=True).str.strip()

# This is making U. S. as You. S.
# # #expand contractions
# df['Text'] = df['Text'].apply(lambda x: contractions.fix(x))


In [24]:
#df['Text'][0]

In [172]:
# write_to_file("data/test2.txt")

In [25]:
# we go through the text and concatinating legal case nombers and special names
df['tokens'] = df['Text'].str.split() 

In [174]:
# write_to_file("test1.txt")

In [26]:
new_tokens = []
for case in df['tokens']:
    new_token = []
    case_found = False
    for tokenIdx in range(len(case)):
        new_token.append(case[tokenIdx])
        # Handling cases
        #United States v. Rostenkowski, 59 F. 3d 1291, 1297 (CADC 1995).
        #United States Supreme Court AARON J. SCHOCK v. UNITED STATES(2019) No. 18-406
        if case[tokenIdx] == 'v.':
            case_found = True
        elif case_found and (case[tokenIdx].startswith('No.') or case[tokenIdx][0].islower() or case[tokenIdx][0].isnumeric() or case[tokenIdx-1].lower().startswith('al.')):
            # we need to deal with this
            last_word = new_token.pop()
            castStr = ''
            while len(new_token) > 0 and (new_token[-1] == 'v.' or (new_token[-1].lower() != 'see' and new_token[-1][0].isupper())) :
                castStr =  new_token.pop() + '|' + castStr
            new_token.append(castStr[:-1])
            case_found = False
            new_token.append(last_word)
    new_tokens.append(new_token)
df['tokens'] = new_tokens
# new_tokens

In [28]:
# change all tokens to lower case
df['tokens'] = df['tokens'].apply(lambda x: [item.lower() for item in x])

In [29]:
df["tokens"][0]

['united|states|supreme|court|brown,|acting|warden|v.|davenport(2022)',
 'no.20-826',
 'argued:',
 'october',
 '5,',
 '2021decided:',
 'april',
 '21,',
 '2022',
 'ervine',
 'davenport',
 'was',
 'convicted',
 'of',
 'first-degree',
 'murder',
 'following',
 'a',
 'jury',
 'trial',
 'where,',
 'at',
 'times,',
 'he',
 'sat',
 'shackled',
 'at',
 'a',
 'table',
 'with',
 'a',
 '"privacy',
 'screen."',
 'on',
 'appeal,',
 'he',
 'argued',
 'that',
 'his',
 'conviction',
 'should',
 'be',
 'set',
 'aside',
 'in',
 'light',
 'of',
 'deck|v.|missouri,',
 '544|u.s.|622,',
 'in',
 'which',
 'this',
 'court',
 'held',
 'that',
 'the',
 'fourteenth',
 "amendment's",
 'due',
 'process',
 'clause',
 'generally',
 'forbids',
 'shackling',
 'a',
 'criminal',
 'defendant',
 'at',
 'trial',
 'absent',
 '"a',
 'special',
 'need."',
 'id.,',
 'at',
 '626.',
 'finding',
 'no',
 '"special',
 'need"',
 'articulated',
 'in',
 'the',
 'record,',
 'the',
 'michigan',
 'supreme',
 'court',
 'agreed',
 'that',


In [140]:
# write_to_file("test2.txt")

# Establish base line performance

In [444]:
# # preprocess df['tokens'] using preocess_text function
# data = df[:100]


In [445]:
# df['tokens'] = preprocess_data(data)


In [446]:
# df['tokens'] = tokenize_and_remove_stopwords(df['tokens'])

In [178]:
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
# df['tokens'][0]

In [ ]:
df["tokens"][0]

In [30]:
df['tokens'] = df['tokens'].apply(nltk.pos_tag)
# df['tokens'][0]

KeyboardInterrupt: 

In [108]:
df.to_csv('preprocessed_data.tsv', index=False, sep='\t')


In [181]:
df_tsv = pd.read_csv('preprocessed_data.tsv', sep='\t')

In [183]:
df_tsv['tokens1'] = df_tsv['tokens'].apply(lambda x: [(word.split(", ")[0][1:-1], get_wordnet_pos(word.split(", ")[1][1:-1])) for word in x[2:-2].split('), (')])
# df['tokens'][0]

In [184]:
df_tsv['tokens1'][0]

[('United|States|Supreme|Court|BROWN,|ACTING|WARDEN|v.|DAVENPORT(2022)',
  'UNK'),
 ('No.20-826', 'n'),
 ('Argued:', 'n'),
 ('October', 'n'),
 ('5,', 'UNK'),
 ('2021Decided:', 'UNK'),
 ('April', 'n'),
 ('21,', 'UNK'),
 ('2022', 'UNK'),
 ('Ervine', 'n'),
 ('Davenport', 'n'),
 ('convicted', 'v'),
 ('first-degree', 'a'),
 ('murder', 'n'),
 ('following', 'v'),
 ('jury', 'n'),
 ('trial', 'n'),
 ('where,', 'n'),
 ('times,', 'n'),
 ('sat', 'v'),
 ('shackled', 'v'),
 ('table', 'a'),
 ('"privacy', 'n'),
 ('screen."', 'n'),
 ('On', 'UNK'),
 ('appeal,', 'n'),
 ('argued', 'v'),
 ('conviction', 'n'),
 ('set', 'v'),
 ('aside', 'r'),
 ('light', 'a'),
 ('Deck|v.|Missouri,', 'n'),
 ('544|U.S.|622,', 'UNK'),
 ('Court', 'n'),
 ('held', 'v'),
 ('Fourteenth', 'n'),
 ("Amendment's", 'n'),
 ('Due', 'n'),
 ('Process', 'n'),
 ('Clause', 'n'),
 ('generally', 'r'),
 ('forbids', 'v'),
 ('shackling', 'v'),
 ('criminal', 'a'),
 ('defendant', 'a'),
 ('trial', 'n'),
 ('absent', 'n'),
 ('"a', 'n'),
 ('special', 'a'),


In [64]:
lemma = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda x: [lemma.lemmatize(word, tag) if tag else lemma.lemmatize(word) for word, tag in x])
# df['tokens'][0]

In [65]:
df['tokens'] = [' '.join(map(str, l)) for l in df['tokens']]

In [ ]:
df["tokens"][0]

In [67]:
df.to_csv('final_preprocessed_data.csv', index=False)

In [24]:
#vectorize the text using tfidf vectorizer
# X_data = vectorize_tfidf(df['tokens'])
tfidf=TfidfVectorizer(min_df = 50, max_df=0.95, ngram_range = (1,3), max_features=1500, norm='l2')
X_data = tfidf.fit_transform(df['tokens'])
df['tokens'] = X_data.toarray()

In [25]:
X_data.shape

(100, 434)

In [72]:
! pip install gensim

     --------------------------------------- 23.9/23.9 MB 36.4 MB/s eta 0:00:00
  Using cached smart_open-6.2.0-py3-none-any.whl (58 kB)
  Using cached Cython-0.29.28-py2.py3-none-any.whl (983 kB)


In [26]:
import gensim.downloader as api
from gensim.models import Word2Vec

In [27]:
w2v_model = Word2Vec(df['tokens'], vector_size=300, window=5, min_count=10)

In [28]:
def create_word2vec_embeddings(X):
    embeddings = []
    for sentence in X:
        sentence_embedding = np.zeros(300)
        for word in sentence:
            if word in w2v_model.wv:
                sentence_embedding += w2v_model.wv[word]
        embeddings.append(sentence_embedding)
    return np.array(embeddings)

In [29]:
w2v_vectors = create_word2vec_embeddings(df['tokens'])

In [30]:
w2v_vectors.shape

(100, 300)

## TF-IDF Model

In [31]:
import gensim
import gensim.downloader as api

def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

def doc2vec_vectors(X):
    documents = list(tagged_document(X[0].split()))
    model = gensim.models.doc2vec.Doc2Vec(documents, vector_size=300, window=5, min_count=10)
    return np.array([model.infer_vector(doc.split()) for doc in X])

In [32]:
doc2vec = doc2vec_vectors(df['tokens'])

In [33]:
#find most similar cases using cosine similarity and tfidf vectors
def find_similar_cases(X, case_id, n):
    pairwise_similarities=np.dot(X,X.T)
    most_similar = pairwise_similarities[case_id].argsort()[:-n-1:-1]
    return most_similar

#find most similar cases using cosine similarity and word2vec vectors

find_similar_cases(X_data, 0, 4)


array([0, 5, 4, 8], dtype=int64)

## Word2Vec Model

In [34]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
#find the max sequence length of word2vec vectors\
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['tokens'])
tokenized_documents=tokenizer.texts_to_sequences(df['tokens'])
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=5000,padding='post')
vocab_size=len(tokenizer.word_index)+1


(5000,)


In [ ]:
w2v_vectors = create_word2vec_embeddings(df['tokens'])

## BERT Model

In [41]:
! pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(df['tokens'])

     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 5.5/5.5 MB 8.8 MB/s eta 0:00:00
     -------------------------------------- 167.2/167.2 MB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 11.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 10.2 MB/s eta 0:00:00
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
     -------------------------------------- 151.6/151.6 kB 8.8 MB/s eta 0:00:00
  Using cached tokenizers-0.13.1-cp39-cp39-win_amd64.whl (3.3 MB)
     ---------------------------------------- 2.5/2.5 MB 9.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125930 sha256=695baba5210bbbaf5df1b9d4bb5ab3754c05bb66f78afacf770b8cb2529f2050
  Stored in directory: c:\users\a

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

similar = cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[:]
)
ind = np.argsort(similar[0])[::-1][:5]

In [43]:
ind

array([ 0, 98,  2,  7, 88], dtype=int64)